In [1]:
import torch
from torch import nn
import requests
import os
from pathlib import Path
URL = 'https://download.pytorch.org/models/resnet18-5c106cde.pth'

In [2]:
%load_ext autoreload

%autoreload 2

In [13]:
from glasses.nn.models.classification.resnet import ResNet, ResNetBasicBlock
from glasses.nn.models.classification.senet import ResNet, SEResNet

from glasses.nn.models.classification.se import SEModule


class SENetBasicBlock(ResNetBasicBlock):
    def __init__(self, in_features: int, out_features: int, reduction: int = 16, *args, **kwargs):
        super().__init__(in_features, out_features, *args, **kwargs)
        self.block.add_module('se', SEModule(out_features))

        
ResNet.resnet18(block=SENetBasicBlock)

ResNet(
  (encoder): ResNetEncoder(
    (gate): Sequential(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
      (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (blocks): ModuleList(
      (0): ResNetLayer(
        (block): Sequential(
          (0): SENetBasicBlock(
            (block): Sequential(
              (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (act1): ReLU(inplace=True)
              (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (se): SEModule(
           

In [5]:
from glasses.nn.models.classification.inception import InceptionABlock

x = torch.rand(1,32, 8, 8)
block = InceptionABlock(32, conv=)
block(x)
# block

InceptionABlock(
  (blocks): ModuleList(
    (0): Sequential(
      (conv1): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
      (conv2): Conv2d(64, 96, kernel_size=(3, 3), stride=(1, 1))
      (conv3): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1))
    )
    (1): Sequential(
      (conv1): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
      (conv2): Conv2d(64, 96, kernel_size=(3, 3), stride=(1, 1))
    )
    (2): Sequential(
      (conv): Conv2d(32, 96, kernel_size=(1, 1), stride=(1, 1))
    )
  )
)